In [ ]:
---
layout: post
title: The Case For Decentralization
description: What is decentralization? And why would I use it over centralized services?
permalink: /digital-famine/microblog/decentralization
breadcrumb: true
---

## Introduction

You can't and *shouldn't* trust massive, centralized tech conglomerates at all. The truth is that using any massive tech company and trusting them with any of your data is a 99.9% of the time a bad move. There's **2 main arguments** for this:

1. Big tech companies collect a lot of data about *you*, that comes with its own plethora of problems; namely, the main problem is the amount of tracking they include in their profit models. See, Google and Meta (for example) make most of their money off of you by brokering your data (collecting and selling it) and using it to target you with... advertisements! The modern society is all *consume*, *consume*, *consume*, and they make a HUGE amount of money off of it. As such, your data is incredibly valuable, and these companies have a [very common](https://haveibeenpwned.com/) tendency to leak that data. Since these centralized models focus on surveillance and selling your data (if that isn't a massive concern already as it paves way for profits over humanity), it can also be really easily abused to control people, bringing us to the next point;

2. Because basically everybody you know uses these centralized platforms, they're tied into said systems and thus reliant on a frankly parasitic relationship, making a perfect mass surveillance model. Governments and companies alike have abused this model to monitor, oppress, and control people in clear violation of legal and constitutional protections. Here's some links:
   - [Electronic Frontier Foundation](https://www.eff.org/)
   - [Global surveillance Wikipedia](https://en.wikipedia.org/wiki/Global_surveillance)
   - [Vault 7 (CIA Hacking Tools Revealed)](https://wikileaks.org/ciav7p1/)
   - [Edward Snowden's Memoir](https://avalonlibrary.net/ebooks/Edward%20Snowden%20-%20Permanent%20Record.pdf)
   - [Edward Snowden's Wikipedia](https://en.wikipedia.org/wiki/Edward_Snowden)

## Trust is a Funny Thing

Here's an example. Assume that I'm a massive tech conglomerate (like Google) and you want to store your files in a very convenient service which saves your files in cloud infrastructure (like Google Drive). Now, using my proprietary service, you can't really know how I'm handling your data, you just have to place your trust in me. (Side note: you can't even trust open source services at times because you don't always know if their servers run that specific code or different code, but in most cases it is *much* better than using large, centralized services.) Now, let's assume that I use some insecure form of encryption, like `AES 128 ECB`.

You can see penguins through it!

![ECB Penguin](https://miro.medium.com/v2/resize:fit:1400/1*zw8Juc6NoZheJt4k61tuEQ.png)


<h3>AES ECB Demo (WASM)</h3>

<label>Key (text, 16 bytes):</label>
<input id="keytext" value="YELLOW SUBMARINE" size="40" />
<br />

<label>Key (hex, readonly):</label>
<input id="keyhex" value="" size="40" readonly />
<br />

<label>Attacker input (string):</label>
<input id="attacker" value="" size="40" />
<br />

<label>Unknown (secret to recover):</label>
<input id="unknown" value="secret-message" size="40" />
<br />

<button id="run">Run demo</button>

<h3>Output</h3>
<pre id="out"></pre>

<script type="module">
import init, { run_ecb_demo } from 'https://the-remakers.github.io/rust-wasm-libs/pkg/examples/examples.js';

async function main() {
    await init('https://the-remakers.github.io/rust-wasm-libs/pkg/examples/examples_bg.wasm');

    const runBtn = document.getElementById('run');
    const outEl = document.getElementById('out');
    const keyTextEl = document.getElementById('keytext');
    const keyHexEl = document.getElementById('keyhex');

    function uint8ArrayToHex(u8) {
        return Array.from(u8).map(b => b.toString(16).padStart(2,'0')).join('');
    }

    function uint8ArrayToString(u8) {
        try { return new TextDecoder().decode(u8); }
        catch(e){ return uint8ArrayToHex(u8); }
    }

    function stringToUint8Array(s) {
        return new TextEncoder().encode(s);
    }

    function updateHexFromText() {
        try {
            keyHexEl.value = uint8ArrayToHex(stringToUint8Array(keyTextEl.value));
        } catch (e) {
            keyHexEl.value = '';
        }
    }

    // Keep the hex display in sync with the text key
    keyTextEl.addEventListener('input', updateHexFromText);
    updateHexFromText();

    runBtn.addEventListener('click', () => {
        outEl.textContent = 'Running...';
        try {
            const keyU8 = stringToUint8Array(keyTextEl.value);
            const attacker = document.getElementById('attacker').value;
            const unknown = document.getElementById('unknown').value;
            if (keyU8.length !== 16) { outEl.textContent='Key must be 16 bytes'; return; }

            const result = run_ecb_demo(keyU8, attacker, unknown);
            const ct = result.ciphertext;
            const rec = result.recovered;
            const steps = Array.from(result.steps || []);

            let out = '';
            out += `ciphertext (${ct.length} bytes):\n${uint8ArrayToHex(ct)}\n\n`;
            out += `recovered (${rec.length} bytes):\n${uint8ArrayToString(rec)}\n\n`;
            out += 'steps:\n';
            for (const s of steps) out += ` - ${s}\n`;
            outEl.textContent = out;
        } catch (e) {
            outEl.textContent = 'Error: '+(e.message||e);
            console.error(e);
        }
    });
}

main();
</script>

### Cracking Your Data

This'll be a pretty technical section, so you're free to skip if you want.


I had an encryption oracle (a black box that, given attacker-controlled input `P`) that returns
```raw
C = Encrypt_K(P || S)
```
where `S` is a fixed secret appended by the server and `K` is a fixed but unknown key. The server used AES128 in ECB mode.

I exploited these properties of ECB:
- AES ECB encrypts each 16 byte block independently:
    ```raw
    C_i = E_K(P_i)
    ```
    where `P_i` is the `i`-th plaintext block. There is no chaining or randomization (IV/nonce).
- Identical plaintext blocks map to identical ciphertext blocks. That determinism leaks structure and enables chosen plaintext recovery when I (the attacker) controls part of the plaintext.

The process is relatively simple:
1. Detect block size `b` (here `b` is `16`) by sending inputs of increasing length and finding the increase in ciphertext length per `b` bytes.
2. Confirm ECB by repeating a block of length `b` and seeing repeated ciphertext blocks.
3. To recover the first unknown byte of `S`, I send an input of length `b-1` of attack bytes. The oracle encrypts `A || S[0..]`, so the first block is `A || S[0]`. I then build a dictionary of all `256` possible last bytes: for each guess `x`, I compute `Encrypt_K(A || x)` and match against the oracle output. The matching ciphertext gives `x = S[0]`.
4. Then you just slide one byte and repeat to recover subsequent bytes.

## But What Does That Have To Do With Centralization?

The attack you and I just performed was simple and general. If a provider offers *any* deterministic server-side encryption with attacker controlled input, they leak secrets. Now, this is a pretty narrow case, but I'll try to expand here a little.

1. You cannot assume the service you're using uses the cryptography you think it does. A proprietary server could use weak encryption (AES128 ECB), reuse IVs, or leak data in a plethora of ways. Open source helps but does not guarantee the deployed binary matches the source.
2. Centralized providers hold keys, metadata, and control service logic. Compromise yields *mass exposure* likewise. They are attractive targets for coercion or surveillance (legal orders, secret warrants, data requests, infiltration, or -- as the current US government likes to do -- no warrants and unconstitutionally!)
3. Many large companies monetize behavioral data; the presence of that business model changes the risk metrics. Data aggregation + surveillance = capability to manipulate, censor, or profile and it's a *very* lucrative venture. There is no incentive to stop after all.
4. Operational mistakes happen. Misconfiguration, poor crypto, migrations, etc. etc. The attack surface is HUGE. All the big tech companies you probably use (Google, Meta, AT&T, Verizon, etc.) have all had data leaks or, for example, the NSA is explicitly tapped in to their traffic (PRISM, Upstream Collection).

### They Can Work Though!

There's loads of things you can do to give your clients pretty good security and privacy; you could use E2EE, split-knowledge or threshold schemes, reproducible builds, remote attestation, stripping metadata, etc. The reason these massive, centralized tech companies don't do this is because they lose out on a lot of money made from monetizing spying and surveillance.

## The Case For Decentralization

The goals here are simple: we want to reduce single points of control, increase auditability, and return data sovereignty. Decentralization is basically splitting up the service and distributing the power originally held by singular corporate entities.

### Pros

You get pretty good benefits!
- No single corporate gatekeeper decides policies, collects data, or holds master keys.
- Endpoints/services can be designed to hold only the minimum required info; users can choose hosts or self-host.
- Federation reduces single failures; the network continues when one node is down
- Standards enable multiple independent implementations (e.g., Matrix Protocol, the Fediverse)
- Projects can be community run, privacy first, or user funded which reduces incentives of surveillance

### Cons

As with anything, it's not all sunshine and rainbows.
- Centralized platforms win because everyone is there. Federation requires cross server moderation which has turned out to be complex. The simple solution, of course, would be to convince people to get on there, but that's hard (first sentence).
- Federation complicates moderation/
- Key management and account recovery are harder. This could be solved through social recovery, hardware keys, or threshold schemes (but these increase attack surface).
- Decentralized storage and replication strategies need careful design to match centralized SLAs.
- Self-hosting is hard for most users.
- And of course, there's no money in freedom! Who's going to fund and support all of this to compete against the tech giants?

## Takeaways

Always be vigilant. Always act carefully and don't be too generous with your trust. I recommend [privacyguides.org](https://www.privacyguides.org/) as a directory for tools to get started with privacy, and -- as always -- [Signal](https://signal.org/#signal) is almost always the answer!

### Your Code

Oh, and for the vault code, you'll need this info:

```raw
/--- BEGIN KEY ---/

meow meoww mroww

/---- END KEY ----/
```


```raw
/--- BEGIN MESSAGE ---/

23753cbf670d9359a33deec7e40d4f42b292ec74c88182d27a32234959392e5b2f9eb7dd3bc8ddb24041cd2f65fdd0b6521de14a2f4be8096897f4100ca05d5dbd0cd0bc1c7bcb45de81320e7eb346d4

/---- END MESSAGE ----/
```